In [1]:
# Importing the necessary packages
import pandas as pd
import numpy as np
import keras
from sklearn.preprocessing import StandardScaler

In [2]:
# Load pima indians dataset
dataset = np.loadtxt("D:\DATA_science\Data_sets\\pima-indians-diabetes.data.csv", delimiter = ',')

In [3]:
X = dataset[:, 0:8]
Y = dataset[:, 8]

In [4]:
# Standardization
a = StandardScaler()
a.fit(X)
X_standardized = a.transform(X)

In [5]:
pd.DataFrame(X_standardized).describe()

,0,1,2,3,4,5,6,7
count,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02
mean,-7.748432e-17,3.614007e-18,-1.327244e-17,7.762888e-17,-5.493291e-17,2.972738e-15,1.924387e-15,2.192980e-16
std,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00
min,-1.141852e+00,-3.783654e+00,-3.572597e+00,-1.288212e+00,-6.928906e-01,-4.060474e+00,-1.189553e+00,-1.041549e+00
25%,-8.448851e-01,-6.852363e-01,-3.673367e-01,-1.288212e+00,-6.928906e-01,-5.955785e-01,-6.889685e-01,-7.862862e-01
50%,-2.509521e-01,-1.218877e-01,1.496408e-01,1.545332e-01,-4.280622e-01,9.419788e-04,-3.001282e-01,-3.608474e-01
75%,6.399473e-01,6.057709e-01,5.632228e-01,7.190857e-01,4.120079e-01,5.847705e-01,4.662269e-01,6.602056e-01
max,3.906578e+00,2.444478e+00,2.734528e+00,4.921866e+00,6.652839e+00,4.455807e+00,5.883565e+00,4.063716e+00


#### Tuning of Hyperparameters : -Batch Size and Epochs

In [5]:
# Importing the necessary packages
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

In [6]:
# Create Model
def create_model():
    model = Sequential()
    model.add(Dense(12, input_dim = 8, activation = "relu"))
    model.add(Dense(8, activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))
    
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

In [8]:
# Create the model
model = KerasClassifier(build_fn = create_model, verbose = 0)
#Define thr GridSearch parameters
batch_size = [10, 20, 40]
epochs = [10, 50, 100]
# Make the dictionary of the gridsearch parameters
param_grid = dict(batch_size = batch_size, epochs = epochs )
# Build and fit the gridsearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(), verbose = 10)
grid_result = grid.fit(X_standardized, Y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START batch_size=10, epochs=10....................................
[CV 1/5; 1/9] END ..................batch_size=10, epochs=10; total time=   8.1s
[CV 2/5; 1/9] START batch_size=10, epochs=10....................................
[CV 2/5; 1/9] END ..................batch_size=10, epochs=10; total time=   2.9s
[CV 3/5; 1/9] START batch_size=10, epochs=10....................................
[CV 3/5; 1/9] END ..................batch_size=10, epochs=10; total time=   3.1s
[CV 4/5; 1/9] START batch_size=10, epochs=10....................................
[CV 4/5; 1/9] END ..................batch_size=10, epochs=10; total time=   2.5s
[CV 5/5; 1/9] START batch_size=10, epochs=10....................................
[CV 5/5; 1/9] END ..................batch_size=10, epochs=10; total time=   2.5s
[CV 1/5; 2/9] START batch_size=10, epochs=50....................................
[CV 1/5; 2/9] END ..................batch_size=10

In [11]:
# Summarize the result
print('Best : {}, using : {}'.format(grid_result.best_score_, grid_result.best_params_) )
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stddev, params in zip(means, stds, params):
    print('{}, {}, with {}'.format(mean, stddev, params))

Best : 0.7669892311096191, using : {'batch_size': 10, 'epochs': 10}
0.7669892311096191, 0.02816910166004853, with {'batch_size': 10, 'epochs': 10}
0.7370257258415223, 0.033172596932143854, with {'batch_size': 10, 'epochs': 50}
0.7317715048789978, 0.04594819312210114, with {'batch_size': 10, 'epochs': 100}
0.7630931258201599, 0.03741421380459523, with {'batch_size': 20, 'epochs': 10}
0.7513369083404541, 0.03670296774669643, with {'batch_size': 20, 'epochs': 50}
0.7278923749923706, 0.03428907465554477, with {'batch_size': 20, 'epochs': 100}
0.7644257783889771, 0.04670263155539899, with {'batch_size': 40, 'epochs': 10}
0.7474577903747559, 0.03315250225048427, with {'batch_size': 40, 'epochs': 50}
0.7266276359558106, 0.031856095029813285, with {'batch_size': 40, 'epochs': 100}


#### Tuning of Hyperparameters : Learning Rate & Dropout Rate 

In [17]:
from keras.layers import Dropout

# Defining a model

def create_model(learning_rate, dropout_rate):
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, input_dim = 8, kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation = 'sigmoid'))
    
    adam = Adam(lr = learning_rate)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# Create model

model = KerasClassifier(build_fn = create_model, verbose = 0, batch_size = 40, epochs = 10)

# Define GridSerchCV parameters

learning_rate = [0.001, 0.01, 0.1]
dropout_rate = [0.0, 0.1, 0.2]

# make a dictionary of GridSearch Parameters

param_grids = dict(learning_rate = learning_rate, dropout_rate = dropout_rate)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model, param_grid = param_grids, cv = KFold(), verbose = 10)
grid_result = grid.fit(X_standardized, Y)



Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 1/5; 1/9] END .....dropout_rate=0.0, learning_rate=0.001; total time=   1.8s
[CV 2/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 2/5; 1/9] END .....dropout_rate=0.0, learning_rate=0.001; total time=   2.0s
[CV 3/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 3/5; 1/9] END .....dropout_rate=0.0, learning_rate=0.001; total time=   1.7s
[CV 4/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 4/5; 1/9] END .....dropout_rate=0.0, learning_rate=0.001; total time=   1.7s
[CV 5/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 5/5; 1/9] END .....dropout_rate=0.0, learning_rate=0.001; total time=   1.9s
[CV 1/5; 2/9] START dropout_rate=0.0, learning_rate=0.01........................
[CV 1/5; 2/9] END ......dropout_rate=0.0, learnin

In [21]:
# Summarize the result
print('Best : {}, Using : {}'.format(grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
std = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, params in zip(means, std, params):
    print("{}, {}, with : {}".format(means, std, params))

Best : 0.7683048963546752, Using : {'dropout_rate': 0.0, 'learning_rate': 0.01}
[0.72800272 0.7683049  0.74219507 0.72801969 0.74883287 0.76563959
 0.73698328 0.75790679 0.73572702], [0.06257947 0.03655624 0.01805759 0.06566303 0.06058967 0.04440064
 0.06532381 0.04488983 0.0293632 ], with : {'dropout_rate': 0.0, 'learning_rate': 0.001}
[0.72800272 0.7683049  0.74219507 0.72801969 0.74883287 0.76563959
 0.73698328 0.75790679 0.73572702], [0.06257947 0.03655624 0.01805759 0.06566303 0.06058967 0.04440064
 0.06532381 0.04488983 0.0293632 ], with : {'dropout_rate': 0.0, 'learning_rate': 0.01}
[0.72800272 0.7683049  0.74219507 0.72801969 0.74883287 0.76563959
 0.73698328 0.75790679 0.73572702], [0.06257947 0.03655624 0.01805759 0.06566303 0.06058967 0.04440064
 0.06532381 0.04488983 0.0293632 ], with : {'dropout_rate': 0.0, 'learning_rate': 0.1}
[0.72800272 0.7683049  0.74219507 0.72801969 0.74883287 0.76563959
 0.73698328 0.75790679 0.73572702], [0.06257947 0.03655624 0.01805759 0.0656630

#### Tuning of Hyperparameters : Activation function and Kernel Initializer

In [28]:
# Defining the model

def create_model(activation_func, init):
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer = init, activation = activation_func))
    model.add(Dropout(0.1))
    model.add(Dense(4, input_dim = 8, kernel_initializer = init, activation = activation_func))
    model.add(Dropout(0.1))
    model.add(Dense(1, input_dim = 8, kernel_initializer = init, activation = activation_func))
    
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# Create model

model = KerasClassifier(build_fn = create_model, verbose = 0, batch_size = 40, epochs = 10)

# Define GridSearchCv Parameters
activation_func = ['sigmoid', 'relu', 'softmax', 'tanh','linear']
init = ['normal', 'zero', 'uniform']

# Make a dictionary of GridSearch Parameters

param_grids = dict(activation_func = activation_func, init = init)

# Build and fit the GridSearchCv
grid = GridSearchCV(estimator = model, param_grid = param_grids, cv = KFold(), verbose = 10)
grid_result = grid.fit(X_standardized, Y)

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV 1/5; 1/15] START activation_func=sigmoid, init=normal.......................
[CV 1/5; 1/15] END .....activation_func=sigmoid, init=normal; total time=   2.0s
[CV 2/5; 1/15] START activation_func=sigmoid, init=normal.......................
[CV 2/5; 1/15] END .....activation_func=sigmoid, init=normal; total time=   1.7s
[CV 3/5; 1/15] START activation_func=sigmoid, init=normal.......................
[CV 3/5; 1/15] END .....activation_func=sigmoid, init=normal; total time=   1.7s
[CV 4/5; 1/15] START activation_func=sigmoid, init=normal.......................
[CV 4/5; 1/15] END .....activation_func=sigmoid, init=normal; total time=   1.9s
[CV 5/5; 1/15] START activation_func=sigmoid, init=normal.......................
[CV 5/5; 1/15] END .....activation_func=sigmoid, init=normal; total time=   1.7s
[CV 1/5; 2/15] START activation_func=sigmoid, init=zero.........................
[CV 1/5; 2/15] END .......activation_func=sigmoi

[CV 1/5; 11/15] END .........activation_func=tanh, init=zero; total time=   2.0s
[CV 2/5; 11/15] START activation_func=tanh, init=zero...........................
[CV 2/5; 11/15] END .........activation_func=tanh, init=zero; total time=   1.8s
[CV 3/5; 11/15] START activation_func=tanh, init=zero...........................
[CV 3/5; 11/15] END .........activation_func=tanh, init=zero; total time=   1.7s
[CV 4/5; 11/15] START activation_func=tanh, init=zero...........................
[CV 4/5; 11/15] END .........activation_func=tanh, init=zero; total time=   2.0s
[CV 5/5; 11/15] START activation_func=tanh, init=zero...........................
[CV 5/5; 11/15] END .........activation_func=tanh, init=zero; total time=   1.8s
[CV 1/5; 12/15] START activation_func=tanh, init=uniform........................
[CV 1/5; 12/15] END ......activation_func=tanh, init=uniform; total time=   1.7s
[CV 2/5; 12/15] START activation_func=tanh, init=uniform........................
[CV 2/5; 12/15] END ......ac

In [29]:
# Summarize the result
print('Best : {}, using : {}'.format(grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : 0.6511586427688598, using : {'activation_func': 'sigmoid', 'init': 'normal'}
0.6511586427688598,0.05244526932680711 with: {'activation_func': 'sigmoid', 'init': 'normal'}
0.6511586427688598,0.05244526932680711 with: {'activation_func': 'sigmoid', 'init': 'zero'}
0.6511586427688598,0.05244526932680711 with: {'activation_func': 'sigmoid', 'init': 'uniform'}
0.6511586427688598,0.05244526932680711 with: {'activation_func': 'relu', 'init': 'normal'}
0.6511586427688598,0.05244526932680711 with: {'activation_func': 'relu', 'init': 'zero'}
0.6511586427688598,0.05244526932680711 with: {'activation_func': 'relu', 'init': 'uniform'}
0.34884136319160464,0.05244525153100685 with: {'activation_func': 'softmax', 'init': 'normal'}
0.34884136319160464,0.05244525153100685 with: {'activation_func': 'softmax', 'init': 'zero'}
0.34884136319160464,0.05244525153100685 with: {'activation_func': 'softmax', 'init': 'uniform'}
0.6511586427688598,0.05244526932680711 with: {'activation_func': 'tanh', 'init'

#### Tuning of hyperparameter : Number of neurons in activation layer

In [31]:
# Defining the model

def create_model(neuron1,neuron2):
    model = Sequential()
    model.add(Dense(neuron1,input_dim = 8,kernel_initializer = 'uniform',activation = 'tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(neuron2,input_dim = neuron1,kernel_initializer = 'uniform',activation = 'tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model

# Create the model

model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 10)

# Define the grid search parameters

neuron1 = [4,8,16]
neuron2 = [2,4,8]

# Make a dictionary of the grid search parameters

param_grids = dict(neuron1 = neuron1,neuron2 = neuron2)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X_standardized,Y)


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 1/5; 1/9] END ......................neuron1=4, neuron2=2; total time=   1.7s
[CV 2/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 2/5; 1/9] END ......................neuron1=4, neuron2=2; total time=   1.7s
[CV 3/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 3/5; 1/9] END ......................neuron1=4, neuron2=2; total time=   2.0s
[CV 4/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 4/5; 1/9] END ......................neuron1=4, neuron2=2; total time=   1.9s
[CV 5/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 5/5; 1/9] END ......................neuron1=4, neuron2=2; total time=   1.9s
[CV 1/5; 2/9] START neuron1=4, neuron2=4........................................
[CV 1/5; 2/9] END ......................neuron1=4

In [32]:
# Summarize the results
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : 0.7670316576957703, using {'neuron1': 8, 'neuron2': 8}
0.7435446977615356,0.04436083314499531 with: {'neuron1': 4, 'neuron2': 2}
0.7474492907524108,0.03102281121656955 with: {'neuron1': 4, 'neuron2': 4}
0.7514132976531982,0.04944150836546482 with: {'neuron1': 4, 'neuron2': 8}
0.7500976204872132,0.04213904249815624 with: {'neuron1': 8, 'neuron2': 2}
0.7591885209083558,0.032952452416745916 with: {'neuron1': 8, 'neuron2': 4}
0.7670316576957703,0.04559504692250687 with: {'neuron1': 8, 'neuron2': 8}
0.7605211853981018,0.04371606684350427 with: {'neuron1': 16, 'neuron2': 2}
0.7656990170478821,0.032190517847094424 with: {'neuron1': 16, 'neuron2': 4}
0.764408802986145,0.0339742488931518 with: {'neuron1': 16, 'neuron2': 8}


#### training of model with optimumm values of hyperparameter


In [34]:
from sklearn.metrics import classification_report, accuracy_score

# Defining the model

def create_model():
    model = Sequential()
    model.add(Dense(16,input_dim = 8,kernel_initializer = 'uniform',activation = 'tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(4,input_dim = 16,kernel_initializer = 'uniform',activation = 'tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(lr = 0.001) #sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model

# Create the model

model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 10)

# Fitting the model

model.fit(X_standardized,Y)

# Predicting using trained model

y_predict = model.predict(X_standardized)

# Printing the metrics
print(accuracy_score(Y,y_predict))

C:\Users\LENOVO\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.78125
